In [60]:
# Coneccion entre Google Colab y el Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **<font color="#ff5653">Funciones</font>**

En este notebook se creara el archivo donde las funciones tomaran la información.

Archivos utilizados:
 - steam_games_ETL.csv
 - users_items_ETL.csv

Funciones que tomaran la información:
- Play_Time_Genre
- User_For_Genre

Archivos creados:
- df_f1y2.csv
- df_f1y2.parquet

In [ ]:
# Libreria
import pandas as pd
import numpy as np
import re                    # Extraer año
import ast

In [ ]:
# DataFrame: steam_games
df_steam_games = pd.read_csv("/content/drive/MyDrive/Datasets_Steam/steam_games_ETL.csv")

In [ ]:
# DataFrame: users_items
df_users_items = pd.read_csv("/content/drive/MyDrive/Datasets_Steam/users_items_ETL.csv")

### 1- "año_lanzamiento"

Necesitamos extraer de df_steam_games de la columna "release_date" (fecha de lanzamiento), el año de lanzamiento.

Y vamos a crear un nuevo DataFrame llamado "año_lanzamiento" con solo una columna que posea el año.

 La lista "año_lanzamiento" la unimos al DataFrame steam_games.

In [ ]:
# Suma de valores nulos de la columna "release_date"
df_steam_games['release_date'].isna().sum()

2067

In [ ]:
# Funcion para extraer el año
def año_lanzamiento(fecha):
    if pd.notna(fecha):
        años = re.search(r'\b\d{4}\b', fecha)
        if años:
            return años.group()
    return np.NaN

In [ ]:
# Llamamos a la funcion: solo para la columna "release_date"
año_lanzamiento = df_steam_games["release_date"].apply(año_lanzamiento)

In [ ]:
año_lanzamiento.head(5)

0    2018
1    2018
2    2017
3    2017
4     NaN
Name: release_date, dtype: object

In [ ]:
# Concatenamos la columna "año_lanzamiento" con el DataFrame df_steam_games
df_steam_games_año = pd.concat([df_steam_games, pd.DataFrame({"año_lanzamiento": año_lanzamiento})], axis=1)

In [ ]:
df_steam_games_año.head(2)

### 2- "horas de juego por genero"

En el dataframe df_users_items tenemos la columna "item_id" donde hay una ID para cada genero.

Por lo que si se suman los datos de la columna "Playtime_forever" agrupados por los ID de la columna "item_id" tendremos las horas de juego para cada genero.

In [ ]:
# Agrupamos los ID similar de las columnas "item_id" y "user_id", para despues se suman las horas de la columna "playtime_forever" de cada columna agrupada
columnas_df_users_items = df_users_items.groupby(["item_id", "user_id"])["playtime_forever"].sum().reset_index()

In [ ]:
columnas_df_users_items.head(2)

,item_id,user_id,playtime_forever
0,10.0,-GM-Dragon,14.0
1,10.0,-KillZone-,987.0


###3- Unión de dataframe

Unimos los dataframe "df_steam_games_año" y "columnas_df_users_items", asi lo usamos en las funciones.

In [ ]:
# Cambiamos el nombre de la columna a "id" a "item_id"
df_steam_games_año.rename(columns={"id": "item_id"}, inplace=True)

In [ ]:
df_steam_games_año.head(2)

In [ ]:
# Unimos ambos dataframe por medio de la columna "item_id"
steam_y_users = pd.merge(columnas_df_users_items, df_steam_games_año, on='item_id', how='inner')

In [ ]:
steam_y_users.head(2)

### 4- Eliminamos columnas del nuevo dataframe

Del dataframe steam_y_users solo nos quedamos con las columnas necesarias para las funciones.

- user_id
- item_id
- title
- año_lanzamiento
- genres
- playtime_forever

In [ ]:
# Elegimos del dataframe steam_y_users las columnas de interes y creamos un nuevo dataframe df_steam_y_users
columnas = ["user_id","item_id","title","genres","año_lanzamiento","playtime_forever"]
df_steam_y_users = steam_y_users[columnas]

In [ ]:
df_steam_y_users.head(2)

,user_id,item_id,title,genres,año_lanzamiento,playtime_forever
0,-GM-Dragon,10.0,Counter-Strike,['Action'],2000,14.0
1,-KillZone-,10.0,Counter-Strike,['Action'],2000,987.0


In [ ]:
# Suma de valores nulos
df_steam_y_users.isna().sum()

In [ ]:
# Eliminamos valores nulos del dataframe
df_steam_y_users.dropna(inplace=True)

In [ ]:
# Eliminamos valores 0 de la columna "playtime_forever"
df2_2 = df_steam_y_users["playtime_forever"]!= 0.0
df_steam_y_users = df_steam_y_users[df2_2]

In [ ]:
df_steam_y_users

In [ ]:
# Ordenamos el indice del dataframe
df_steam_y_users.reset_index(drop=True, inplace=True)

- Agrupamos dos dataframe y obtuvimos un solo dataframe steam_y_users.
- Este dataframe tenia 4.244.812 filas y 16 columnas.
- Se le realizo a este dataframe un ETL.
- El nuevo dataframe df_steam_y_users posee 2.763.977 filas y 6 columnas.

### 5- Cambio de tipos de datos

Cambiamos el tipo de datos que tienen algunas columnas.

In [ ]:
# Informacion de las columnas y el tipo de datos que tienen
df_steam_y_users.info()

In [ ]:
# Columna "title" cambiamos de tipo de dato object a string
df_steam_y_users["title"] = df_steam_y_users["title"].astype("string")

In [ ]:
# Columna "año_lanzamiento" cambiamos de tipo de dato object a int64
df_steam_y_users["año_lanzamiento"] = df_steam_y_users["año_lanzamiento"].astype("int64")

In [ ]:
df_steam_y_users.info()

In [ ]:
item_id_x2 = df_steam_y_users["item_id"].duplicated()
df_steam_y_users[item_id_x2]

### 6- Eliminamos columnas

Para las funciones PlayTimeGenre y UserForGenre, necesitamos las columnas user_id, año_lanzamiento, genres y playtime_forever.

In [ ]:
# Eliminamos las columnas "item_id" y "title"
df_funcion_1y2 = df_steam_y_users.drop(["item_id","title"], axis=1)


In [ ]:
df_funcion_1y2.head(2)

In [ ]:
# Agrupamos las columnas user_id, año_lanzamiento y genres. Luego, calcula la suma de la columna playtime_forever para cada grupo.
df_funcion_1y2_suma = df_funcion_1y2.groupby(["user_id", "año_lanzamiento", "genres"])["playtime_forever"].sum().reset_index()

In [ ]:
df_funcion_1y2_suma.head(3)

In [ ]:
# Funcion conversion a lista
def convertir_a_lista(x):
    try:
        # Eliminar corchetes, dividir por ', ' y encerrar cada elemento entre comillas simples
        elementos = x.strip('[]').split(', ')
        elementos_formateados = [f"'{elemento}'" for elemento in elementos]
        return elementos_formateados
    except Exception as e:
        # Manejar cualquier excepción que pueda ocurrir durante la conversión
        return []

In [ ]:
# Aplicamos la funcion a la columna "genres"
df_funcion_1y2_suma["genres"] = df_funcion_1y2_suma["genres"].apply(convertir_a_lista)

In [ ]:
# Separamos los genero
df_funcion_1y2_suma = df_funcion_1y2_suma.explode("genres")

In [ ]:
# Agrupamos
df_f1y2 = df_funcion_1y2_suma.groupby(["user_id", "año_lanzamiento"]).agg({"genres": list, "playtime_forever": "sum"}).reset_index()

In [ ]:
df_f1y2.head(2)

In [ ]:
def transformar(genres):
    return [element.strip("'") for element in genres]


In [ ]:
# Llamamos a la funcion
df_f1y2["genres"] = df_f1y2["genres"].apply(transformar)

In [ ]:
df_f1y2.head(2)

,user_id,año_lanzamiento,genres,playtime_forever
0,--000--,2006,"[Indie, Simulation]",1850.0
1,--000--,2009,[Action],5329.0


In [ ]:
# Guardamos el dataframe obtenido en formato csv y parquet
df_f1y2.to_csv("df_f1yf2.csv", index=False)

In [ ]:
df_f1y2.to_parquet("df_f1yf2.parquet", index=False)